In [ ]:
import numpy as np

In [ ]:
f1 = lambda x: x**2
f2 = lambda x: x-2

In [ ]:
period = 4
x = np.linspace(0, 4, 100)
maxN = 10
minN = 2
fArr = f1(x)

In [ ]:
#coeffs
cosCoeff = np.zeros(10)
sinCoeff = np.zeros(10)